### 卷积神经网络

#### 卷积层

卷积神经网络（convolutional neural network）是含有卷积层（convolutional layer）的神经网络。

在二维互相关运算中，卷积窗口从输入数组的最左上方开始，按从左往右、从上往下的顺序，依次在输入数组上滑动。当卷积窗口滑动到某一位置时，窗口中的输入子数组与核数组按元素相乘并求和，得到输出数组中相应位置的元素。

##### 多通道卷积
<img src = "img/conv_multi_in.svg">

##### 1X1的多输入多输出通道卷积
<img src="img/conv_1x1.svg">

#### 池化层

为了缓解卷积层对位置的过度敏感性。

    同卷积层一样，池化层每次对输入数据的一个固定形状窗口（又称池化窗口）中的元素计算输出。不同于卷积层里计算输入和核的互相关性，池化层直接计算池化窗口内元素的最大值或者平均值。该运算也分别叫做最大池化或平均池化。在二维最大池化中，池化窗口从输入数组的最左上方开始，按从左往右、从上往下的顺序，依次在输入数组上滑动。当池化窗口滑动到某一位置时，窗口中的输入子数组的最大值即输出数组中相应位置的元素。
<img src="img/conv_1x1.svg">

池化层也有填充、多通道等特性

### 实战CNN系列

### LeNet

在之前的MLP模型中，我们知道它最大的缺点是当层数增加时，模型参数急剧增加。CNN的出现一方面是为了解决这个参数爆炸的问题，另一方面，单纯将二维的像素点拍平成一位数据，可能会造成邻近的像素的信息丢失。因为图像在同一列邻近的像素在这个向量中可能相距较远。它们构成的模式可能难以被模型识别。

##### LeNet模型

LeNet分为卷积层块和全连接层块两个部分。

    卷积层块里的基本单位是卷积层后接最大池化层：卷积层用来识别图像里的空间模式，如线条和物体局部，之后的最大池化层则用来降低卷积层对位置的敏感性。卷积层块由两个这样的基本单位重复堆叠构成。在卷积层块中，每个卷积层都使用5×5的窗口，并在输出上使用sigmoid激活函数。第一个卷积层输出通道数为6，第二个卷积层输出通道数则增加到16。这是因为第二个卷积层比第一个卷积层的输入的高和宽要小，所以增加输出通道使两个卷积层的参数尺寸类似。卷积层块的两个最大池化层的窗口形状均为2×2，且步幅为2。由于池化窗口与步幅形状相同，池化窗口在输入上每次滑动所覆盖的区域互不重叠。

    卷积层块的输出形状为(批量大小, 通道, 高, 宽)。当卷积层块的输出传入全连接层块时，全连接层块会将小批量中每个样本变平（flatten）。也就是说，全连接层的输入形状将变成二维，其中第一维是小批量中的样本，第二维是每个样本变平后的向量表示，且向量长度为通道、高和宽的乘积。全连接层块含3个全连接层。它们的输出个数分别是120、84和10，其中10为输出的类别个数。

#### LeNet实现

In [1]:
#  basic package
import numpy as np
import keras
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D,Flatten

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# 导入数据
# fashion-mnist
def load_mnist(path, kind='train'):
    import os
    import gzip

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

X_train, y_train = load_mnist('F:/机器学习/动手学深度学习/data/fashion', kind='train')
X_test, y_test = load_mnist('F:/机器学习/动手学深度学习/data/fashion', kind='t10k')

In [3]:
X_train.shape

(60000, 784)

In [4]:
%%time
# 处理数据
from keras.utils.np_utils import to_categorical


X_train_1 = X_train / 255
x_train = X_train_1.reshape(-1, 28, 28, 1)

X_test_1 = X_test / 255
x_test = X_test_1.reshape(-1, 28, 28, 1)

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

Wall time: 268 ms


In [5]:
x_train.shape

(60000, 28, 28, 1)

In [6]:
%%time
# 定义模型keras

batch_size = 256

model = Sequential([
    # 卷积层块
    Conv2D(filters=6, kernel_size=(5, 5), activation='sigmoid', input_shape=(28, 28, 1)),  # 使用 6 个大小为 5x5 的卷积滤波器。
    MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'),
    Conv2D(filters=16, kernel_size=(5, 5), activation='sigmoid'),
    MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'),
    
    Flatten(),
    # 全连接层块
    Dense(120,activation='sigmoid'),
    Dense(84,activation='sigmoid'),
    Dense(10, activation='softmax')
])

# compile
sgd = SGD(lr=0.9)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# train
model.fit(x_train, y_train, batch_size=32, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 16s 264us/step - loss: 2.3063 - acc: 0.1000
Epoch 2/5
60000/60000 [==============================] - 14s 226us/step - loss: 2.3043 - acc: 0.1007
Epoch 3/5
60000/60000 [==============================] - 13s 224us/step - loss: 2.3041 - acc: 0.1012
Epoch 4/5
60000/60000 [==============================] - 14s 225us/step - loss: 2.3037 - acc: 0.1018
Epoch 5/5
60000/60000 [==============================] - 14s 227us/step - loss: 2.3032 - acc: 0.1030
Wall time: 1min 10s


In [16]:
loss_and_metrics = model.evaluate(x_test, y_test, batch_size=256)
print(loss_and_metrics)

10000/10000 [==============================] - 0s 42us/step
[0.5787755011558533, 0.7773]
